In [ ]:
pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 65.1 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
import jax.numpy as jnp
import jax

# Enable JAX 64-bit precision (required for pulse-level control)
jax.config.update("jax_enable_x64", True)


In [ ]:
# Define 2-qubit device
num_qubits = 2
dev = qml.device("default.mixed", wires=range(num_qubits))

# Define qubit frequencies (in GHz, matching the red laser frequency)
qubit_freq = [5.0, 5.1]  # Example frequencies

# Create base Hamiltonian for our system
# The connections list specifies qubit couplings
connections = [(0, 1)]  # Connection between qubits 0 and 1
coupling = [0.01]       # Coupling strength of 10 MHz

# Base Hamiltonian representing the system dynamics
H0 = qml.pulse.transmon_interaction(
    qubit_freq=qubit_freq,
    connections=connections,
    coupling=coupling,
    wires=range(num_qubits)
)

# Polarizing filter implementation
# Create drive Hamiltonians (analogous to the polarizing filter in the diagram)
duration = 15  # Duration of the pulse in nanoseconds

def polarizing_filter(p, t):
    # Ensure parameter is always an array, not a scalar
    p_array = jnp.atleast_1d(p)
    return qml.pulse.pwc(duration)(p_array, t)

# Drive Hamiltonian for qubit 0 - analogous to laser through polarizing filter
H_drive0 = qml.pulse.transmon_drive(
    amplitude=polarizing_filter,
    phase=0,
    freq=qubit_freq[0],
    wires=[0]
)


In [ ]:
# Half waveplate with 90-degree rotation implementation
def half_waveplate_rotation(p, t):
    # Ensure parameter is always an array, not a scalar
    p_array = jnp.atleast_1d(p)
    return qml.pulse.pwc(duration)(p_array, t)

# Create drive Hamiltonian for the computational stage
H_compute0 = qml.pulse.transmon_drive(
    amplitude=half_waveplate_rotation,
    phase=jnp.pi/2,  # 90-degree phase for rotation
    freq=qubit_freq[0],
    wires=[0]
)

# For the Deutsch-Jozsa algorithm, we need an interaction between qubits
# This implements the oracle part shown in the diagram
def interaction_pulse(p, t):
    # Ensure parameter is always an array, not a scalar
    p_array = jnp.atleast_1d(p)
    return qml.pulse.pwc(duration)(p_array, t)

H_interaction = qml.pulse.transmon_drive(
    amplitude=interaction_pulse,
    phase=0,
    freq=(qubit_freq[0] + qubit_freq[1])/2,  # Intermediate frequency
    wires=[0, 1]
)


In [ ]:
# Define the complete quantum circuit with all three stages
@qml.qnode(dev, interface='jax')
def deutsch_jozsa_circuit(params_prep, params_compute, params_interact, t_total):
    # State preparation (laser + polarizing filter)
    qml.evolve(H0 + H_drive0)(params_prep, t=duration)

    # Computation (half waveplate with 90-degree rotation)
    qml.evolve(H0 + H_compute0)(params_compute, t=duration)

    # Interaction for the oracle functionality
    qml.evolve(H0 + H_interaction)(params_interact, t=duration)

    # Measurement of both qubits (corresponds to the detector in the diagram)
    return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]


In [ ]:
# Define noise parameters
t1 = [50000, 50000]  # T1 relaxation times in ns
t2 = [30000, 30000]  # T2 dephasing times in ns

# Create noisy device
dev_noisy = qml.device("default.mixed", wires=range(num_qubits))


# Define the noisy circuit
@qml.qnode(dev_noisy, interface='jax')
def noisy_deutsch_jozsa_circuit(params_prep, params_compute, params_interact, t_total):
    # State preparation with polarizing filter
    qml.evolve(H0 + H_drive0)(params_prep, t=duration)

    # Apply noise channels after state preparation
    for i in range(num_qubits):
        qml.AmplitudeDamping(duration / t1[i], wires=i)
        qml.PhaseDamping(duration / t2[i], wires=i)

    # Computation with half waveplate
    qml.evolve(H0 + H_compute0)(params_compute, t=duration)

    # Apply noise after computation
    for i in range(num_qubits):
        qml.AmplitudeDamping(duration / t1[i], wires=i)
        qml.PhaseDamping(duration / t2[i], wires=i)

    # Interaction for oracle functionality
    qml.evolve(H0 + H_interaction)(params_interact, t=duration)

    # Apply noise after interaction
    for i in range(num_qubits):
        qml.AmplitudeDamping(duration / t1[i], wires=i)
        qml.PhaseDamping(duration / t2[i], wires=i)

    return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]


In [ ]:
# Initialize parameters for each stage with requires_grad=True
params_prep = jnp.array([0.5])       # Amplitude for state preparation
params_compute = jnp.array([0.3])    # Amplitude for computation
params_interact = jnp.array([0.2])   # Amplitude for interaction (oracle)
# Total evolution time
t_total = 3 * duration

# Execute the circuit
result = noisy_deutsch_jozsa_circuit(params_prep, params_compute, params_interact, t_total)
print("Measurement results:", result)


Measurement results: [Array(0.89973854, dtype=float64), Array(0.55995379, dtype=float64)]


In [ ]:
# Define cost function based on desired output
def cost(params):
    params_prep, params_compute, params_interact = params
    result = noisy_deutsch_jozsa_circuit(params_prep, params_compute, params_interact, t_total)
    # For Deutsch-Jozsa, we want qubit 0 to be -1 for balanced function
    return ((result[0] + 1)**2)  # Cost is 0 when result[0] = -1

# Initial parameters
init_params = [jnp.array([0.5]), jnp.array([0.3]), jnp.array([0.2])]

# Optimize using gradient descent
opt = qml.GradientDescentOptimizer(stepsize=0.1)
params = init_params

for i in range(20):
    params = opt.step(cost, params)
    if i % 5 == 0:
        print(f"Step {i}: cost = {cost(params):.6f}")


/usr/local/lib/python3.11/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Step 0: cost = 3.609007
Step 5: cost = 3.609007
Step 10: cost = 3.609007
Step 15: cost = 3.609007
